In [ ]:
%%writefile consts.py

import numpy as np         # arrays    
import sounddevice as sd   # modulo de conexión con portAudio
import soundfile as sf     # para lectura/escritura de wavs
import matplotlib.pyplot as plt # dibujos

SRATE, CHUNK = 48000, 1024

# Modelos de **Síntesis** digital


# Síntesis analógica/digital

**Sintetizador**: dispositivo electrónico
de generación de sonido ...

- Antes de los ordenadores:

    osciladores analógicos, Telharmonium (síntesis aditiva), órgano Hammond, *Theremin*, ondas Martenot, (síntesis sustractiva), sintetizador Moog (síntesis modular).. 

---

- Después llegaron los ordenadores, síntesis digital:

    $\leadsto$ algoritmos para la generación de muestras de sonido


    - Emulación de las técnicas de los sintes analógicos precedentes.

    - pero además... nuevas posibilidades

    Prophet-5, Yamaha GS-1 (síntesis FM),  **Yamaha DX7** (FM), Roland D-50 (*sampler*), ...

# Síntesis aditiva

- Idea: hemos visto que una nota de guitarra, flauta o piano está compuesta por frecuencias, múltiplos de una fundamental (armónicos)

    $\leadsto$ recomponer una nota musical por *adición de frecuencias* (armónicos) $f_1,\ldots,f_k$ con amplitudes dadas $r_1,\ldots,r_k$ (idea del antiguo Teleharmonium y el órgano Hammond)

<center>
<img src="aditiva.png" width="500" />
</center>


- Cada una de estas frecuencias se genera con un oscilador independiente y se suman después.

    También pueden incluirse frecuencias inarmónicas!



# Modelando síntesis aditiva en Python

- recuperamos la clase oscilador (versión simple)

In [ ]:
%%writefile osc.py

from consts import *
import numpy as np         

class Osc:
    def __init__(self,freq=440.0,amp=1.0,phase=0.0):
        self.freq = freq
        self.amp = amp
        self.phase = phase
        self.frame = 0

    def next(self):    
        out = self.amp*np.sin(2*np.pi*(np.arange(self.frame,self.frame+CHUNK))*self.freq/SRATE)
        self.frame += CHUNK
        return out

    def setFreq(self,freq):
        self.freq = freq

    def getFreq(self):
        return self.freq

## Mezclador de señal

- Recibe $n$ señales $signals=[s_0,...,s_{n-1}]$

- y las suma para generar una nueva señal

  ... pero reduce la amplitud para no saturar la señal

  Una forma sencilla de *equilibrar la intensidad* de la señal resultante (*equal power*) es multiplicar la suma por $\frac{1}{\sqrt(n)}$

In [ ]:
%%writefile mixer.py

from consts import *
import numpy as np         

class Mixer:
    def __init__(self):
        self.signals = []

    def addSignal(self,osc):
        self.signals.append(osc)

    def next(self):
        out = np.zeros(CHUNK)
        for s in self.signals:
            out += s.next()       
        # normalización de energía para mantener volumen percibido
        return out/np.sqrt(len(self.signals))

## Síntesis aditiva

In [ ]:
from consts import *
from osc import Osc   
from mixer import *

# stream de salida con callBack
master = None
def callback(outdata, frames, time, status):
    if status: print(status)
    if master:    
        s = master.next()
        s = np.float32(s)
    else:
        s = np.zeros(CHUNK,dtype=np.float32)
    outdata[:] = s.reshape(-1, 1)
# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, callback=callback, blocksize=CHUNK)
stream.start()


freq = 110  # frec base
step = 5    # variacion de la frecuencia base
amps = [0.2, 0.15, 0.15, 0.15, 0.2, 0.1, 0.1]  # amps de los armónicos

freqs = [freq*(i+1) for i in range(len(amps))]  # frecs de los armónicos

# osciladores armónicos
oscs = [Osc(freqs[i],amps[i]) for i in range(len(freqs))]


mixer = Mixer()
for osc in oscs:
    mixer.addSignal(osc)

master = mixer

end =  False

while not end:
    c = input("[F/f] subir/bajar frequencia  [q] salir: ")
    if c=='q': 
        end = True
    else:
        if c=='F':
            freq = freq+step
        elif c=='f':
            freq = freq-step
        freqs = [freq*(i+1) for i in range(len(amps))]  # freqs de los armónicos
        for i in range(len(amps)): 
            oscs[i].setFreq(freqs[i])

        print(f"\rfreqs:",end='')
        for i in range(len(amps)): 
            print(f"F{i}: {freqs[i]} v{i}: {amps[i]}   ",end='')

stream.stop()

# Enriqueciendo timbre
- Con desfases en los armónicos
- Ligera desafinación de armónicos



Matemáticamente la síntesis aditiva es:

$$s(t)=\sum_{k=1}^{N}\underbrace{A_k}_{vol_k} sin(2\pi \underbrace{f_k}_{armonico_k}t+\underbrace{\theta_k}_{desfase})$$

- Además $sin$ puede reemplazarse/combinarse con cuadrada, triangular,...


In [ ]:
from consts import *
from osc import Osc
from mixer import *


# stream de salida con callBack
master = None
def callback(outdata, frames, time, status):
    if status: print(status)
    if master:    
        s = master.next()
        s = np.float32(s)
    else:
        s = np.zeros(CHUNK,dtype=np.float32)
    outdata[:] = s.reshape(-1, 1)
# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, callback=callback, blocksize=CHUNK)
stream.start()


freq = 110  # freq base
step = 5
amps = [0.3, 0.2, 0.15, 0.15, 0.1, 0.1, 0.2, 0.1, 0.5, 0.3]
phases = [0, 0.04, 0.1, 0.07, 0.03, 0.02, 0, 0, 0, 0, 0]
    

freqs = [freq*(i+1) for i in range(len(amps))]  # frecs de los armónicos
oscs = [Osc(freqs[i],amps[i],phases[i]) for i in range(len(freqs))]


mixer = Mixer()
for osc in oscs:
    mixer.addSignal(osc)

master = mixer



end =  False
while not end:
    c = input("[F/f] subir/bajar frecuencia  [q] salir: ")
    if c=='q': 
        end = True
    else:
        if c=='F':
            freq = freq+step
        elif c=='f':
            freq = freq-step
        freqs = [freq*(i+1) for i in range(len(amps))]  # frecs de los armónicos
        for i in range(len(amps)): 
            oscs[i].setFreq(freqs[i])

        print(f"\rFrecs:",end='')
        for i in range(len(amps)): 
            print(f"F{i}: {freqs[i]} v{i}: {amps[i]}   ",end='')


    
stream.stop()

# Gráficamente


In [ ]:
from consts import *

%matplotlib inline


end = int(SRATE/50)
pi = np.pi
t = np.arange(0,end)

#tabla de armonicos
arms = [(1.,100.,-pi/2),
    (0.5,300.,0),
    (0.25,500.,-pi/2),
    (0.1,700.,0),
    (0.05,800.,0),
    (0.01,900.,-pi/2)]

s = np.zeros(end)
plt.subplot(211)
plt.title("Tabla de parciales aislados")
plt.xlim(0,end)
for n in arms:
    a,f,ph = n
    p = a*np.cos(2*pi*f*t/SRATE + ph)
    s += p
    plt.plot(t, p)
plt.subplot(212)
plt.title("Onda resultante")
plt.xlim(0,end)
plt.plot(t, s)
plt.tight_layout()
plt.show()

# Este modelo ya se utilizaba en sintetizadores analógicos


<center>
<img src="Moog-Modular-Series.jpg" width="400" />
</center>


<center>
<img src="moog_minimoogd.jpg" width="400" />
<img src="minimoog.jpg" width="400" />
</center>


# *Contras* de la síntesis aditiva

- En general es costosa: 
    - para obtener timbres ricos se necesitan muchos osciladores (10 o más)
    - muchos parámetros que controlar...
    - para *variar* los sonidos en el tiempo hay que controlar cada uno de esos parámetros


# Síntesis sustractiva

Parte de un sonido y se *esculpe* el timbre buscado

- Se parte de una señal muy rica en armónicos
    - En rigor, se parte de *ruido blanco* (contiene todas las frecuencias)
    - pero pueen utlizarse otras señalse de partida (onda cuadrada, diente de sierra...)

-   Se utilizan filtros para *recortar* o **esculpir** el contenido armónico y obtener un timbre determinado
        - Suelen utilizarse filtros LP resonantes

<center>
<img src="media/subtractive1.png" width="500"/>
<img src="media/subtractive2.png" width="400"/>
</center>


Ejemplo del proceso en
<https://en.wikipedia.org/wiki/Subtractive_synthesis>


# Refinando el modelo aditivo: síntesis FM

La síntesis aditiva trata de emular la generación de armónicos de un instrumento natural.

- Pero en la naturaleza, estos armónicos no son señales periódicas puras (las cuerdas nunca están perfectamente equilibradas).

    $\leadsto$ hay pequeñas variaciones continuas de frecuencia = **modulaciones en frecuencia**

Este es el "fundamento" de la síntesis por Frecuencia Modulada (FM)... aunque en realidad fue un hallazgo casual:

- *The Synthesis of Complex Audio Spectra by Means of Frequency Modulation*, John Chowning, 1973

$\leadsto$ Yamaha DX7


# Idea de la síntesis FM 


<center>
<img src="fm.png" width="600" />
</center>


Matemáticamente:
$${sin(2\pi f_c+\beta \underbrace{sin(2\pi f_m))}_{\textrm{moduladora}}}$$

-   $f_c$ frecuencia portadora (**carrier**) $\leadsto$ pitch de la nota.

-   $f_m$ frecuencia moduladora (**modulator**) $\leadsto$ vibrato

-   $\beta$ índice de modulación (modulator **index**)


# Y si añadimos más moduladores?

Y si añadimos más moduladores?, i.e., que la portadora juegue el papel de moduladora (de otra portadora): y si anidamos más senos dentro de los senos?

Supongamos

$$\begin{array}{r}
    sin(2\pi f_c+\beta sin(2\pi f_m))~~\\
    sin(2\pi f_{c'} + \beta' sin(2\pi f_c+\beta sin(2\pi f_m)))~\\
    sin(2\pi f_{c''} + \beta'' sin(2\pi f_{c'} + \beta' sin(2\pi f_c+\beta sin(2\pi f_m)))~\\        
  \end{array}$$

Otra formulación (mismo resultado). Supongamos:

$$[\underbrace{(f_0,vol)}_{carrier},\underbrace{(f_1,\beta_1),(f_2,\beta_2),\ldots,(f_n,\beta_n)}_{moduladoras~ (anidadas)}]$$


# Mas extensiones? Timbres?

- Se pueden utilizar/combinar otros tipos de onda y efectos.

**La pregunta del millón**: ¿Que combinaciónes de modulares y que parámtros $(f,\beta)$ utilizo para obtener un oboe o un piano?

$\leadsto$ experimentar, coleccionar presets y meter envolventes ADSR!

Una pista:

<http://javelinart.com/FM_Synthesis_of_Real_Instruments.pdf>

Un sintetizador:

<https://pypi.org/project/synthplayer/>


# Yamaha DX7

<center>
<img src="YAMAHA_DX7.jpg" width="800" />
</center>



<center>
<img src="algsFM.png" width="800" />
</center>

Un DX7 virtual: https://asb2m10.github.io/dexed/


# Otros modelos de síntesis. Wavetable (tabla de ondas)

- Nuestros osciladores calculan una y otra vez los valores del seno (para cada muestra)

- Podemos tabular un ciclo del seno y reutilizamos los valores para ganar eficiencia

$\leadsto$ *wavetable lookup synthesis*

**Ojo**, mucha confusión terminológica con *wavetable*:

- En algunos sitios presentan una versión ingénua de wavetable:

    - Se copia repetidamente una muestra hasta rellenar un CHUNK

    - Pero qué ocurre si cambiamos la frecuencia sobre la marcha?

        $\leadsto$ recálculo de tabla de ondas: nueva tabla de ondas!!

    - Además...el cambio de frecuencia durante la ejecución produce una discontinuidad en los CHUNKS generados $\leadsto$ POPs


# Wavetable (look up) "real"

Se utiliza la misma tabla para todas las frecuencias:

Idea: recorrer cíclicamente la (misma) muestra almacenada en la tabla, *variando la velocidad de reproducción*, en función de la frecuencia deseada:

- Si repetimos al doble de velocidad subimos una octava

- $\leadsto$ podemos generar cualquier frecuencia con la misma tabla


Ventajas:

-   Mayor eficiencia: siempre una misma tabla de ondas

-   Continuidad en los CHUNKs (no pops)
    

In [ ]:
from IPython.display import Video
Video("media/waveTable.mp4")

In [ ]:
%%writefile oscWaveTable.py

from consts import *

class OscWaveTable:
    # size: tamaño del ciclo de onda
    # cuanto mayor el tamaño, mayor la resolución en frecuencia
    def __init__(self, frec, vol, size):
        self.frec = frec
        self.vol = vol
        self.size = size
        # un ciclo completo de seno en [0,2pi)
        t = np.linspace(0, 1, num=size)
        self.waveTable = np.sin(2 * np.pi * t)
        # arranca en 0
        self.fase = 0
        # paso en la wavetable en funcion de frec y RATE
        self.step = self.size/(SRATE/self.frec)

    def setFrec(self,frec): 
        self.frec = frec
        self.step = self.size/(SRATE/self.frec)

    def getFrec(self): 
        return self.frec    

    def next(self):
        samples = np.zeros(CHUNK,dtype=np.float32)
        cont = 0
        
        while cont < CHUNK:
            self.fase = (self.fase + self.step) % self.size

            # con truncamiento, sin redondeo
            # samples[cont] = self.waveTable[int(self.fase)]

            # con redondeo
            #x = round(self.fase) % self.size
            #samples[cont] = self.waveTable[x]
                        
            # con interpolacion lineal                                    
            x0 = int(self.fase) % self.size
            x1 = (x0 + 1) % self.size
            y0, y1 = self.waveTable[x0], self.waveTable[x1]            
            samples[cont] = y0 + (self.fase-x0)*(y1-y0)/(x1-x0)

            cont = cont+1
    
        return self.vol*samples


### Veamos que no hay pops gráficamente

In [ ]:
from consts import *

osc = OscWaveTable(110,1,1024)

s = np.empty(0,dtype=np.float32)
for i in range(4):
    s = np.concatenate([s,osc.next()])

plt.plot(s)


osc.setFrec(120)
s = np.empty(0,dtype=np.float32)
for i in range(4):
    s = np.concatenate([s,osc.next()])


plt.plot(s)
plt.show()


# Reproducimos sonido

In [ ]:
from consts import *
from oscWaveTable import OscWaveTable

# stream de salida con callBack
master = None
def callback(outdata, frames, time, status):
    if status: print(status)
    if master:    
        s = master.next()
        s = np.float32(s)
    else:
        s = np.zeros(CHUNK,dtype=np.float32)
    outdata[:] = s.reshape(-1, 1)
# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, callback=callback, blocksize=CHUNK)
stream.start()


osc = OscWaveTable(220,1,1024)
master = osc

end = False
while not end:
    c = input(f"freq {osc.getFrec()}  [F/f] subir/bajar  [q] salir\n") 
    if c=='q': end = True
    elif c=='F': osc.setFrec(osc.getFrec()+10)
    elif c=='f': osc.setFrec(osc.getFrec()-10)


stream.close()

# Modelado físico

Utiliza un modelo físico/matemático (ecuaciones) para emular la forma de producción de sonido de los instrumentos. 

Por ejemplo, para una cuerda vibrante, el modelo puede incluir:

-   La generación del sonido propiamente dicho (cuerda vibrante)

-   La evolución en el tiempo de dicho sonido.

-   La simulación del cuerpo del instrumento (resonancia)

-   etc


# Karplus-Strong, 1980s

Un modelo clásico (el primero?) para cuerda pulsada


<center>
<img src="karplus.png" width="600" />
</center>


In [3]:
from IPython.display import Video
Video("ksMovie.mp4")



-   La cuerda "recién pulsada" produce un *caos sonoro* (ruido)

-   Los rebotes tienen un efecto feedback: delay + filtro

-   Y el sonido se va estructurando armónicamente en el tiempo.

Todo esto ocurre muy rápido!



# Implementación Karplus-Strong


In [5]:
from consts import *

def KarplusStrong(frec, dur):
  N = SRATE // int(frec)  # la frecuencia determina el tamanio del buffer
  buf = np.random.rand(N) * 2 - 1  # buffer inicial: ruido

  nSamples = int(dur*SRATE)
  samples = np.empty(nSamples, dtype=float)  # salida

  # generamos los nSamples haciendo recorrido circular por el buffer
  for i in range(nSamples):
      samples[i] = buf[i % N]  # recorrido de buffer circular
      buf[i % N] = 0.5 * (buf[i % N] + buf[(1 + i) % N]) # filtrado
  return samples 


In [11]:
# una nota
stream = sd.OutputStream(samplerate=SRATE,blocksize=CHUNK,channels=1)  
stream.start()
stream.write(np.float32(KarplusStrong(110,12)))
stream.close()   

In [16]:
# una melodía 
stream = sd.OutputStream(samplerate=SRATE,blocksize=CHUNK,channels=1)  
stream.start()

# varias notas
mel = [440, 349.228, 391.995, 261.625, 261.625,  391.995, 440, 349.228]
mel = [m/2 for m in mel]
dur = [1, 1, 1, 2 ]

for i in range(len(mel)):
    stream.write(np.float32(KarplusStrong(mel[i],dur[i%4]/1.5)))

stream.close()   

In [18]:
# resonancia artificial, una reverb por convolución
stream = sd.OutputStream(samplerate=SRATE,blocksize=CHUNK,channels=1)  
stream.start()


from consts import *
import scipy
# convolution
ir, fs = sf.read('st-andrews.wav')
ir = ir.astype(np.float32)

print("\nKarplus Strong convolved")
for i in range(len(mel)):
    nota = KarplusStrong(mel[i],dur[i%4]*1.2)*0.6
    nota = scipy.signal.convolve(nota,ir,mode='valid',method='fft')
    stream.write(np.float32(nota))

stream.close()   


Karplus Strong convolved


ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred



# Referencias de modelado físico

- Karplus Strong online: <http://amid.fish/javascript-karplus-strong>

Mas modelos físicos

- https://ccrma.stanford.edu/software/clm/compmus/clm-tutorials/pm.html#k-s

- Julius Smith, Perry Cook, Chuck language (STK)



# Sampler

Es una **aproximación muy distinta** a las anteriores:

- Todos los modelos anteriores producen sonido púramente sintetizado: generado por el ordenador

- El sampler utiliza *material sonoro real* previamente grabado de un instrumento

    - Muestra de corta duración: normalmente una nota (... no es la idea de los loops y la remezcla de los DJ).

    - Se reproduce a distintas velocidades para producir las diferentes notas.

Los samplers incluyen:

- Manejo de regiones *sustain* para repetir en loop y poder alargar la duración del sonido

Y además suelen incluir:

- envolventes de volumen (ADSR y otras)
- filtros


(Ejemplo piano de la hoja de problemas, renoise, lmms)

# Síntesis granular

Puede verse como una forma especial de sampler:

-   Las muestras se trocean en pequeños fragmentos de muy corta duración (5-100 milisegundos)

-   Se re-agrupan en conjuntos más grandes (de manera síncrona o asíncrona)

Permite crear timbres y **texturas sonoras** de gran comlejidad.

- https://www.hispasonic.com/reviews/5-novedosas-aplicaciones-para-entender-visualmente-sintesis-granular/40850

### Referencias

Un buen punto de entrada:

- http://granularsynthesis.com/books.php